In [40]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from stellargraph import StellarGraph

# Read the CSV file
df = pd.read_csv(r"C:\Github repo\IIT-BHU-Summer-Internship-Network-Security\Flow As Graph\Dataset\mini_multi_dataset\nodes.csv")

# Get the payload ratio and label from the user
payload_ratio = float(input("Enter the payload ratio: "))
label = input("Enter the label: ")

# Create an empty graph
G = nx.Graph()

# Add nodes to the graph
for _, row in df.iterrows():
    node_id = row['nodeid']
    node_feature = row['node_feature']
    G.add_node(node_id, feature=node_feature)

# Add edges to the graph
# for _, row in df.iterrows():
#     source_node = row['source_node_id']
#     destination_nodes = row['destination_node_id']
    
#     for dest_node in destination_nodes:
#         G.add_edge(source_node, dest_node)

# Set the label for the graph
G.graph['label'] = label
# Display the graph
pos = nx.spring_layout(G)  # Define the layout of the nodes
plt.figure(figsize=(10, 6))  # Set the size of the figure
nx.draw_networkx(G, pos=pos, with_labels=True)  # Draw the graph
plt.title(label)  # Set the title of the graph
plt.show()
file_path = r"D:\IIT BHU Intership\graph\test\graph.pkl"

# Save the graph using pickle
with open(file_path, "wb") as file:
    pickle.dump(graph, file)

print("Graph saved successfully.")


ModuleNotFoundError: No module named 'stellargraph'

## 

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GraphConv  # Import GraphConv
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from sklearn.metrics import accuracy_score


# Define your GCN model
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(121, 128)
        self.conv2 = GraphConv(128, 128)
        self.conv3 = GraphConv(128, 128)
        self.dropout = nn.Dropout(0.5)
        self.lin = nn.Linear(128, 9)

    def forward(self, data):
        # Your forward pass code
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = self.conv3(x, edge_index)
        x = F.relu(x)

        x = global_mean_pool(x, data.batch)

        x = self.dropout(x)
        x = self.lin(x)
        x = F.log_softmax(x, dim=1)

        return x

# Rest of your code...


# Initialize the model
model = GCN()

# Define your training parameters
batch_size = 1
learning_rate = 0.01
num_epochs = 10

# Define your training dataset and dataloader
train_dataset = r"D:\IIT BHU Intership\graph\train"
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define your testing dataset and dataloader
test_dataset = "D:\IIT BHU Intership\graph\test"
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_samples = 0

    for batch in train_loader:
        optimizer.zero_grad()
        data = train_dataset  # Move data to GPU if available
        outputs = model(data)
        labels = data.y
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs
        total_samples += data.num_graphs

    # Calculate average training loss for the epoch
    avg_loss = total_loss / total_samples

    # Evaluation on the test set
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for batch in test_loader:
            data = batch.to(torch.device('cuda'))  # Move data to GPU if available
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(data.y.tolist())
            y_pred.extend(predicted.tolist())

    # Calculate accuracy on the test set
    accuracy = accuracy_score(y_true, y_pred)

    # Print training and testing metrics
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")


AttributeError: 'str' object has no attribute 'x'